In [ ]:
from classes.openai_wrapper import OpenAIWrapper
from classes.process_data import PrepData
from classes.rf_regression_optimized import RandomForestRegression
from classes.tune_model import TuneModel
from classes.iterate_model import IterateModel

from dotenv import load_dotenv
import pandas as pd
import logging
import ast
import os

load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Initiate OpenAI Wrapper Class
openai_key = os.getenv("OPENAI_API_KEY")

openai_wrapper = OpenAIWrapper(openai_key)


# Preprocess the dataset based on AI evaluation
dataset = pd.read_csv('../data/Advertising Budget and Sales.csv')

prompt_docs = "../docs/RandomForestRegressorDocs.txt"
performance_tracking_file = "../docs/PerformanceTracking.json"

data_prepper = PrepData(dataset, openai_wrapper, performance_tracking_file)
X, y, label_encoders = data_prepper.preprocess_dataset()


# Perform Random Forest Regression
rf_regressor = RandomForestRegression(
    X=X, 
    y=y, 
    label_encoders=label_encoders
    )

# First round model results:
mse, r2, accuracy, test_size, model_params, feature_importance = rf_regressor.rf_regression()

# Instantiate class for AI-driven model tuning
model_iterator = IterateModel(label_encoders=label_encoders, desired_accuracy=94.5, run_threshold=10, tuner=TuneModel(openai_wrapper, prompt_docs), performance_tracking_file=performance_tracking_file)

model_iterator.iterate_model(X, y, mse, r2, accuracy, test_size, model_params, feature_importance)